# SPR 2026 - TF-IDF + TabPFN

**Notebook para submissão offline no Kaggle.**

---
**CONFIGURAÇÃO OFFLINE:**
1. No Kaggle, vá em Settings → Internet → **OFF**
2. Adicione o dataset: `Prior Labs / TabPFN v2 Weights`

**LIMITAÇÕES DO TABPFN:**
- Máximo 10.000 amostras de treino (usaremos todas se possível)
- Máximo 500 features (usamos TruncatedSVD)
---

In [ ]:
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD
from sklearn.model_selection import cross_val_score
import warnings
warnings.filterwarnings('ignore')

# TabPFN
import sys
sys.path.append('/kaggle/input/tabpfn-v2-weights')
from tabpfn import TabPFNClassifier

SEED = 42
DATA_DIR = '/kaggle/input/spr-2026-mammography-report-classification'
np.random.seed(SEED)
print('Bibliotecas carregadas!')

In [ ]:
# Carregar dados
train = pd.read_csv(f'{DATA_DIR}/train.csv')
test = pd.read_csv(f'{DATA_DIR}/test.csv')

print(f'Train: {train.shape}')
print(f'Test: {test.shape}')

In [ ]:
# TF-IDF Vectorizer
tfidf = TfidfVectorizer(
    max_features=10000,
    ngram_range=(1, 2),
    min_df=2,
    max_df=0.95,
    sublinear_tf=True
)

X_train_tfidf = tfidf.fit_transform(train['report'])
X_test_tfidf = tfidf.transform(test['report'])
y_train = train['target'].values

print(f'TF-IDF shape: {X_train_tfidf.shape}')

In [ ]:
# Redução de dimensionalidade com TruncatedSVD (para TabPFN que aceita max 500 features)
N_COMPONENTS = 100  # TabPFN funciona melhor com menos features

svd = TruncatedSVD(n_components=N_COMPONENTS, random_state=SEED)
X_train = svd.fit_transform(X_train_tfidf)
X_test = svd.transform(X_test_tfidf)

print(f'SVD shape: {X_train.shape}')
print(f'Variância explicada: {svd.explained_variance_ratio_.sum():.4f}')

In [ ]:
# TabPFN - funciona melhor com dados menores, então faremos ensemble se necessário
MAX_TRAIN_SIZE = 10000

if len(X_train) <= MAX_TRAIN_SIZE:
    # Usar todos os dados
    model = TabPFNClassifier(
        device='cuda',
        n_estimators=8,
        random_state=SEED
    )
    
    # Cross-validation
    scores = cross_val_score(model, X_train, y_train, cv=5, scoring='f1_macro')
    print(f'CV F1-Macro: {scores.mean():.5f} (+/- {scores.std():.5f})')
    
    model.fit(X_train, y_train)
    predictions = model.predict(X_test)
else:
    # Ensemble com subsampling estratificado
    from sklearn.model_selection import StratifiedShuffleSplit
    
    n_ensembles = 5
    all_preds = []
    
    splitter = StratifiedShuffleSplit(n_splits=n_ensembles, train_size=MAX_TRAIN_SIZE, random_state=SEED)
    
    for i, (train_idx, _) in enumerate(splitter.split(X_train, y_train)):
        print(f'\nEnsemble {i+1}/{n_ensembles}')
        
        X_subset = X_train[train_idx]
        y_subset = y_train[train_idx]
        
        model = TabPFNClassifier(
            device='cuda',
            n_estimators=8,
            random_state=SEED + i
        )
        
        model.fit(X_subset, y_subset)
        preds = model.predict_proba(X_test)
        all_preds.append(preds)
    
    # Média das probabilidades
    avg_probs = np.mean(all_preds, axis=0)
    predictions = np.argmax(avg_probs, axis=1)

print('\nModelo treinado!')

In [ ]:
# Predições e submissão
submission = pd.DataFrame({
    'ID': test['ID'],
    'target': predictions
})

submission.to_csv('submission.csv', index=False)

print('submission.csv criado!')
print(submission['target'].value_counts().sort_index())